# QASMをQulacsで実行する#

In [74]:
import matplotlib.pyplot as plt
import numpy as np
import time 
import random
from qulacs import QuantumState
from qulacs import QuantumCircuit
from qulacs.gate import DenseMatrix
from qulacs.circuit import QuantumCircuitOptimizer

In [75]:
from qulacs import QuantumState
from qulacs.gate import Identity, X,Y,Z #パウリ演算子
from qulacs.gate import H,S,Sdag, sqrtX,sqrtXdag,sqrtY,sqrtYdag #1量子ビット Clifford演算
from qulacs.gate import T,Tdag #1量子ビット 非Clifford演算
from qulacs.gate import RX,RY,RZ #パウリ演算子についての回転演算
from qulacs.gate import CNOT, CZ, SWAP #2量子ビット演算
from qulacs.gate import U1,U2,U3 #IBM Gate

In [76]:
nqubits = 10
state = QuantumState(10)
state.set_zero_state()
circuit = QuantumCircuit(10)

### QASMのパーサー###
quantum volume benchmarkにあるような、`U1`、`U3`演算と`cx`からなるようなQASMを想定しています。

In [1]:
import re
import parse


def parse_qasm_to_QulacsCircuit(input_filename,qulacs_circuit):

    with open(input_filename, "r") as ifile:
        lines = ifile.readlines()

        for line in lines:
            s = re.search(r"qreg|cx|u3|u1", line)

            if s is None:
                continue

            elif s.group() == 'qreg':
                match = re.search(r"\d\d*", line)
                print(match)
                continue

            elif s.group() == 'cx':
                match = re.findall(r"\[\d\d*\]", line)  # int抽出
                c_qbit = int(match[0].strip('[]'))
                t_qbit = int(match[1].strip('[]'))
                qulacs_circuit.add_gate(CNOT(c_qbit,t_qbit))   

                continue

            elif s.group() == 'u3':
                m_r = re.findall(r"[-]?\d\.\d\d*", line)  # real抽出, 負符号考慮
                m_i = re.findall(r"\[\d\d*\]", line)  # int抽出

                # target_bit = m_i[0]
                # u3parameters = m_r
                qulacs_circuit.add_gate(U3(int(m_i[0].strip('[]')),float(m_r[0]),float(m_r[1]),float(m_r[2])))

                continue

            elif s.group() == 'u1':
                m_r = re.findall(r"[-]?\d\.\d\d*", line)  # real抽出
                m_i = re.findall(r"\[\d\d*\]", line)  # int抽出

                qulacs_circuit.add_gate(U1(int(m_i[0].strip('[]')),float(m_r[0])))

                continue


`"./QvolBench/quantum_volume_n5_d2.qasm"`のとことにQASMのパスを置く

In [82]:
parse_qasm_to_QulacsCircuit("./QvolBench/quantum_volume_n5_d2.qasm",circuit)

<_sre.SRE_Match object; span=(7, 8), match='5'>


きちんと回路が構成されていることを確認しておこう。

In [83]:
print(circuit)

*** Quantum Circuit Info ***
# of qubit: 10
# of step : 28
# of gate : 80
# of 1 qubit gate: 56
# of 2 qubit gate: 24
Clifford  : no
Gaussian  : no




`update`をすると、QASMからパースした回路が実行される

In [84]:
circuit.update_quantum_state(state) 

In [85]:
print(state.get_vector())

[-0.03846929+0.13299097j  0.0646164 +0.18091389j  0.13391629-0.01810708j
 ...  0.        +0.j          0.        +0.j
  0.        +0.j        ]
